In [32]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [41]:
# 어떤 행렬을 정의한다.

x = np.matrix([[1,5,3,4],[2,3,6,2],[3,1,7,8]])

# 5. LU분해

## 1) 기본 개념
### (1) 기본 개념은, 각각 소블럭의 1행과 1열을 외적하여 행렬로 전개한 값을, 원래 행렬에서 지속적으로 빼서
### (2) a11 성분으로 나누어 1행과 1열을 원래 행렬의 값과 동일하게 만들어 준 후에
### (3) 1행과 1열을 0으로 만들어주면서 지속적으로 축소해준다.

In [34]:
# 1열에 1행을 외적할 경우, 생성되는 행렬의 1행과 1열엔 a00성분이 중복해서 들어가게 된다.
# normalizer는 a00로 나눠줌으로서 중복성을 제거해주고, 원래 행렬에서 잔차 행렬을 뺄 때 1행 1열은 0이 되도록 만들어주는 
# 핵심 역할을 담당한다.
normalizer = 1/(x[0,0])

# 1행에 normalizer를 곱해준다.
temp_l = normalizer * x[0].T
# 1열을 임시로 저장해주고
temp_u = x.T[0]
l = normalizer * x[0].T
u = x.T[0]

print(l,u)

[[1.]
 [5.]
 [3.]
 [4.]] [[1 2 3]]


In [35]:
# 1행과 1열을 외적하여 n*m 잔차행렬을 새로 생성해준다.
temp = np.matmul(temp_l,temp_u).T
print(temp)

[[ 1.  5.  3.  4.]
 [ 2. 10.  6.  8.]
 [ 3. 15.  9. 12.]]


In [36]:
# 원래 행렬에서 잔차행렬을 빼 1행과 1열을 0으로 만들어준다.
x = x - temp
print(x)

[[  0.   0.   0.   0.]
 [  0.  -7.   0.  -6.]
 [  0. -14.  -2.  -4.]]


In [37]:
normalizer = 1/x[1,1]
temp_l = normalizer * x[1].T
temp_u = x.T[1]

#위 iteration의 반복이나, i=2부터는 하삼각행렬과 상삼각행렬을 numpy의 stack으로 지속적으로 누적해준다.
l = np.hstack([l,temp_l])
u = np.vstack([u,temp_u])
temp = np.matmul(temp_l,temp_u).T
x = x - temp

In [38]:
normalizer = 1/x[2,2]
temp_l = normalizer * x[2].T
temp_u = x.T[2]
l = np.hstack([l,temp_l])
u = np.vstack([u,temp_u])
temp = np.matmul(temp_l,temp_u).T
x = x - temp
# 최종적인 잔차행렬이 0행렬이 되면 성공이다.
print(x)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [39]:
# 함수로 정의한 LU분해
def LUdecomp(x) :
    if type(x) != type(np.matrix([0])):
        raise ValueError 
    
    #행렬의 행과 열 차원중 작은쪽을 s로 지정해준다.
    s = np.min(x.shape)
    
    normalizer = 1/(x[0,0])
    l = normalizer * x[0].T
    u = x.T[0]
    temp = np.matmul(l,u).T
    x = x - temp
    
    for i in range(1,s):
        normalizer = 1/x[i,i]
        temp_l = normalizer * x[i].T
        temp_u = x.T[i]
        l = np.hstack([l,temp_l])
        u = np.vstack([u,temp_u])
        temp = np.matmul(temp_l,temp_u).T
        x = x - temp
    return (l,u)

In [47]:
x = np.matrix([[1,5,3,4],[2,3,6,2],[3,1,7,8]])

L,U = LUdecomp(x)

print("하삼각행렬")
print(L)
print("상삼각행렬")
print(U)

하삼각행렬
[[ 1.         -0.         -0.        ]
 [ 5.          1.         -0.        ]
 [ 3.         -0.          1.        ]
 [ 4.          0.85714286 -4.        ]]
상삼각행렬
[[  1.   2.   3.]
 [  0.  -7. -14.]
 [  0.   0.  -2.]]
